- ResNet101V2 batch=32: 85%

In [1]:
import os
from matplotlib import pyplot as plt
import pandas as pd
import tensorflow as tf
import numpy as np
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from sklearn import svm
from tqdm import tqdm
from utils.preprocessing import _parse_function, _parse_augmentation_function
import cv2
from efficientnet.tfkeras import EfficientNetB1,EfficientNetB3,EfficientNetB2,EfficientNetB0
from efficientnet.tfkeras import center_crop_and_resize, preprocess_input

In [2]:
datasetDir = "DataP2_crop/Train"
testDir = "DataP2_crop/Dev"

datasetCSV = "DataP2_crop/train.csv"
testCSV = "DataP2_crop/dev.csv"

header = ["ImageName", "Level","x","y","w","h"]
dataset_df = pd.read_csv(datasetCSV, names=header)
test_df = pd.read_csv(testCSV, names=header)

train_tfrecord = "tfrecords/train.tfrecord"
valid_tfrecord = "tfrecords/valid.tfrecord"
test_tfrecord = "tfrecords/test.tfrecord"

In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE # 自動調整模式
batch_size = 32

raw_dataset = tf.data.TFRecordDataset(train_tfrecord)
# dataset = dataset.repeat() # 重复数据集
dataset = raw_dataset.map(_parse_augmentation_function, num_parallel_calls=AUTOTUNE) # 解析数据
dataset = dataset.shuffle(buffer_size=batch_size * 100) # 在缓冲区中随机打乱数据
train_data  = dataset.batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE)

raw_dataset = tf.data.TFRecordDataset(valid_tfrecord)
valid_dataset = raw_dataset.map(_parse_function, num_parallel_calls=AUTOTUNE)
valid_data = valid_dataset.batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE)

raw_dataset = tf.data.TFRecordDataset(test_tfrecord)
test_dataset = raw_dataset.map(_parse_function, num_parallel_calls=AUTOTUNE)
test_data = test_dataset.batch(batch_size=batch_size).prefetch(buffer_size=AUTOTUNE)

In [4]:
# 建立模型儲存目錄
model_dir = 'log/models'
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

In [5]:
# for image, label in dataset.take(100):    
#     plt.title(str(label))
#     plt.imshow(image.numpy())
#     plt.show()
#     print(label)

In [6]:
# 訓練Model(卷積神經網路)
# basemodel = tf.keras.applications.EfficientNetB7(include_top=False, weights=None, input_shape=(224, 224, 3))
basemodel = EfficientNetB0(include_top=False, weights=None, input_shape=(224, 224, 3))

model = tf.keras.Sequential([
        basemodel,
        layers.GlobalAveragePooling2D(),
        layers.Dense(3, activation='sigmoid')])


# model.load_weights("log/models/Best-Mango-Model-ResNet101V2-hinge.h5")


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3843      
Total params: 4,053,407
Trainable params: 4,011,391
Non-trainable params: 42,016
_________________________________________________________________


In [7]:
EPOCHS = 200
learning_rate = 0.01

modelName = "EfficientB0"

# 建立回調函數
log_dir = os.path.join('log', 'EfficientB0')
model_cbk = keras.callbacks.TensorBoard(log_dir=log_dir)
model_esp = keras.callbacks.EarlyStopping(monitor='categorical_accuracy',
                                          patience=30,
                                          mode='max')

model_mckp = keras.callbacks.ModelCheckpoint(model_dir + '/'+modelName+'.h5',
                                             monitor='categorical_accuracy',
                                             save_best_only=True,
                                             model='max')


# 設定優化器、損失函數、指標函數
model.compile(keras.optimizers.Adam(lr=0.001,decay=1e-6),
#               loss=keras.losses.BinaryCrossentropy(),
              loss=tf.keras.losses.CategoricalCrossentropy(), 
              metrics=[keras.metrics.CategoricalAccuracy()])

In [8]:
history = model.fit(train_data,
                    epochs=EPOCHS,
                    validation_data=valid_data,
                    callbacks=[model_cbk, model_esp, model_mckp])

Epoch 1/200
985/985 [==============================] - 316s 321ms/step - loss: 0.7864 - categorical_accuracy: 0.6780 - val_loss: 0.8725 - val_categorical_accuracy: 0.6812
Epoch 2/200
985/985 [==============================] - 307s 312ms/step - loss: 0.7588 - categorical_accuracy: 0.6947 - val_loss: 0.8125 - val_categorical_accuracy: 0.6465
Epoch 3/200
985/985 [==============================] - 296s 301ms/step - loss: 0.7484 - categorical_accuracy: 0.7040 - val_loss: 0.7110 - val_categorical_accuracy: 0.7177
Epoch 4/200
985/985 [==============================] - 301s 306ms/step - loss: 0.7454 - categorical_accuracy: 0.6964 - val_loss: 0.7715 - val_categorical_accuracy: 0.7065
Epoch 5/200
985/985 [==============================] - 295s 300ms/step - loss: 0.7424 - categorical_accuracy: 0.6966 - val_loss: 0.7085 - val_categorical_accuracy: 0.7188
Epoch 6/200
985/985 [==============================] - 294s 298ms/step - loss: 0.7587 - categorical_accuracy: 0.6978 - val_loss: 0.8185 - val_cat

Epoch 49/200
985/985 [==============================] - 288s 293ms/step - loss: 0.4970 - categorical_accuracy: 0.7987 - val_loss: 0.4924 - val_categorical_accuracy: 0.7968
Epoch 50/200
985/985 [==============================] - 289s 293ms/step - loss: 0.4936 - categorical_accuracy: 0.8011 - val_loss: 0.4628 - val_categorical_accuracy: 0.8137
Epoch 51/200
985/985 [==============================] - 286s 290ms/step - loss: 0.4904 - categorical_accuracy: 0.8029 - val_loss: 0.4847 - val_categorical_accuracy: 0.8046
Epoch 52/200
985/985 [==============================] - 288s 293ms/step - loss: 0.4892 - categorical_accuracy: 0.8029 - val_loss: 0.4799 - val_categorical_accuracy: 0.8045
Epoch 53/200
985/985 [==============================] - 288s 292ms/step - loss: 0.4920 - categorical_accuracy: 0.8018 - val_loss: 0.4550 - val_categorical_accuracy: 0.8135
Epoch 54/200
985/985 [==============================] - 285s 290ms/step - loss: 0.4846 - categorical_accuracy: 0.8036 - val_loss: 0.4836 - v

985/985 [==============================] - 290s 295ms/step - loss: 0.3801 - categorical_accuracy: 0.8495 - val_loss: 0.4854 - val_categorical_accuracy: 0.8166
Epoch 144/200
985/985 [==============================] - 290s 294ms/step - loss: 0.3766 - categorical_accuracy: 0.8510 - val_loss: 0.4818 - val_categorical_accuracy: 0.8186
Epoch 145/200
985/985 [==============================] - 292s 296ms/step - loss: 0.3752 - categorical_accuracy: 0.8547 - val_loss: 0.4868 - val_categorical_accuracy: 0.8172
Epoch 146/200
985/985 [==============================] - 292s 296ms/step - loss: 0.3766 - categorical_accuracy: 0.8493 - val_loss: 0.4673 - val_categorical_accuracy: 0.8175
Epoch 147/200
985/985 [==============================] - 290s 294ms/step - loss: 0.3748 - categorical_accuracy: 0.8504 - val_loss: 0.4734 - val_categorical_accuracy: 0.8197
Epoch 148/200
985/985 [==============================] - 292s 296ms/step - loss: 0.3693 - categorical_accuracy: 0.8535 - val_loss: 0.4689 - val_categ

In [9]:
model_dir = 'log/models'
model.load_weights(model_dir + '/'+modelName+'.h5')
loss, acc = model.evaluate(test_data)
print("\nLoss: {}, Accuracy: {}".format(loss, acc))

    219/Unknown - 13s 57ms/step - loss: 0.5979 - categorical_accuracy: 0.7996
Loss: 0.5978756680166069, Accuracy: 0.7995714545249939


In [10]:
preds = []
pred_probability = []
for index, test_name in tqdm(enumerate(test_df['ImageName'])):
    test_image_path = os.path.join(testDir, test_name)
    test_image = Image.open(test_image_path)
    test_image = np.array(test_image)    
    test_image = tf.image.resize(test_image, (224, 224))
    test_image = tf.cast(test_image, tf.float32) / 255.
    test_image = np.expand_dims(test_image, axis=0)
    

    pred = model.predict(test_image)
    pred_probability.append(pred)
    pred = tf.argmax(pred, axis=1).numpy()
    if pred == 0:
        label = 'A'
    elif pred == 1:
        label = 'B'
    elif pred == 2:
        label = 'C'
        
    preds.append(label)
    
test_df['predict'] = preds
test_df

7000it [04:22, 26.63it/s]


,ImageName,Level,x,y,w,h,predict
0,32783.jpg,A,479,268,346,261,A
1,24935.jpg,B,493,282,319,257,B
2,30841.jpg,A,493,189,327,269,A
3,57713.jpg,A,463,258,337,276,A
4,85593.jpg,A,453,290,331,275,B
...,...,...,...,...,...,...,...
6995,64163.jpg,B,155,4,740,707,C
6996,09923.jpg,A,135,31,809,644,C
6997,43250.jpg,A,255,29,709,691,A
6998,01499.jpg,B,153,43,773,677,A


In [11]:
# Color Histogram
# 畫出 RGB 三種顏色的分佈圖
def calcMask(img):
    mask = np.zeros(img.shape[:2], np.uint8)
    top = int(img.shape[0]*0.1)
    bottom = int(img.shape[0]*0.9)
    left = int(img.shape[1]*0.1)
    right = int(img.shape[1]*0.9)

    mask[top:bottom, left:right] = 255
#     mask_img = cv2.bitwise_and(img, img , mask = mask)
    return mask

def RGB_histogram(img):
    mask = calcMask(img)
    color = ('b','g','r')
    for i, col in enumerate(color):
        histr = cv2.calcHist([img],[i],mask,[256],[0, 256])
        plt.plot(histr, color = col)
        plt.xlim([0, 256])
def gray_histogram(img):
    mask = calcMask(img)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hist = cv2.calcHist([gray], [0], mask, [256], [0, 256])
    plt.plot(hist)

In [14]:
#測試資料判別 機率
datasetDir = "DataP2_crop/Train"
testDir = "DataP2_crop/Dev"

datasetCSV = "DataP2_crop/train.csv"
testCSV = "DataP2_crop/dev.csv"

header = ["ImageName", "Level","x","y","w","h"]
dataset_df = pd.read_csv(datasetCSV, names=header)
test_df = pd.read_csv(testCSV, names=header)


plt.figure(figsize=(15, 10))
count = 0
count_abs = 0
for i in range(7000):
    
    if(test_df['Level'][i] != preds[i]):
        print('ImageName:',test_df['ImageName'][i],
              'Label:',test_df['Level'][i])
        print('predict',preds[i])
        
        print("A probability:",pred_probability[i][0][0])
        print("B probability:",pred_probability[i][0][1])
        print("C probability:",pred_probability[i][0][2])
        
        pred_probabil_sort=sorted(pred_probability[i][0])
        if(abs(pred_probabil_sort[-2]-pred_probabil_sort[-1]) < 0.6) :
            count_abs += 1
        count += 1 
#         imgName = test_df['ImageName'][i]
#         imgPath = os.path.join(testDir, imgName)
#         img = plt.imread(imgPath)
#         plt.imshow(img)
#         plt.show()
#         RGB_histogram(img)        
#         plt.show()
        
print('count:',count)
print('count_abs:',count_abs)

ImageName: 85593.jpg Label: A
predict B
A probability: 0.12569162
B probability: 0.30333468
C probability: 0.005029533
ImageName: 66721.jpg Label: A
predict B
A probability: 0.15075442
B probability: 0.25265282
C probability: 0.06387486
ImageName: 68466.jpg Label: A
predict B
A probability: 0.22733794
B probability: 0.5165366
C probability: 0.00539602
ImageName: 08962.jpg Label: B
predict A
A probability: 0.60694873
B probability: 0.082084745
C probability: 0.046393014
ImageName: 50281.jpg Label: C
predict A
A probability: 0.18182833
B probability: 0.10706057
C probability: 0.07468175
ImageName: 63831.jpg Label: A
predict B
A probability: 0.07706006
B probability: 0.68775177
C probability: 0.013508814
ImageName: 66599.jpg Label: B
predict A
A probability: 0.44240436
B probability: 0.36134857
C probability: 0.008407426
ImageName: 26781.jpg Label: A
predict B
A probability: 0.1345454
B probability: 0.46067023
C probability: 0.0013616533
ImageName: 95590.jpg Label: A
predict C
A probabili

predict B
A probability: 0.131638
B probability: 0.30702245
C probability: 0.018014885
ImageName: 94407.jpg Label: A
predict C
A probability: 0.17259845
B probability: 0.046285868
C probability: 0.24963593
ImageName: 26887.jpg Label: A
predict B
A probability: 0.2703356
B probability: 0.6072169
C probability: 0.0024594702
ImageName: 73440.jpg Label: A
predict B
A probability: 0.007397291
B probability: 0.6475585
C probability: 0.0072428538
ImageName: 47011.jpg Label: A
predict B
A probability: 0.009932339
B probability: 0.5266869
C probability: 0.028485699
ImageName: 09931.jpg Label: A
predict B
A probability: 0.14744394
B probability: 0.54437256
C probability: 0.024431404
ImageName: 62413.jpg Label: A
predict B
A probability: 0.08310437
B probability: 0.48711962
C probability: 0.0051272428
ImageName: 99711.jpg Label: A
predict B
A probability: 0.15405108
B probability: 0.58698666
C probability: 0.01328049
ImageName: 79907.jpg Label: A
predict C
A probability: 0.0037980624
B probabilit

C probability: 0.008735317
ImageName: 89933.jpg Label: A
predict B
A probability: 0.04463094
B probability: 0.5505572
C probability: 0.0070988815
ImageName: 61162.jpg Label: A
predict B
A probability: 0.10665917
B probability: 0.3923063
C probability: 0.003445287
ImageName: 90499.jpg Label: A
predict B
A probability: 0.11802836
B probability: 0.2631239
C probability: 0.0028702288
ImageName: 21603.jpg Label: A
predict B
A probability: 0.05204605
B probability: 0.38993633
C probability: 0.06435464
ImageName: 69207.jpg Label: A
predict B
A probability: 0.0007106639
B probability: 0.7790007
C probability: 0.0009191997
ImageName: 61530.jpg Label: A
predict B
A probability: 0.008395921
B probability: 0.79603523
C probability: 0.005327119
ImageName: 96681.jpg Label: B
predict A
A probability: 0.39107016
B probability: 0.3031572
C probability: 0.00075443403
ImageName: 99431.jpg Label: A
predict B
A probability: 0.089301944
B probability: 0.35667148
C probability: 0.008454621
ImageName: 01229.j

ImageName: 11420.jpg Label: A
predict B
A probability: 0.26129958
B probability: 0.48660472
C probability: 0.014331547
ImageName: 68158.jpg Label: A
predict B
A probability: 0.18188006
B probability: 0.48711708
C probability: 0.0010704743
ImageName: 20739.jpg Label: A
predict B
A probability: 0.23342988
B probability: 0.43326005
C probability: 0.0010170722
ImageName: 30936.jpg Label: A
predict B
A probability: 0.2243916
B probability: 0.44815564
C probability: 0.0016002038
ImageName: 90308.jpg Label: B
predict A
A probability: 0.5223234
B probability: 0.21368116
C probability: 0.0071105203
ImageName: 56014.jpg Label: B
predict A
A probability: 0.5307151
B probability: 0.03762479
C probability: 0.019611849
ImageName: 77546.jpg Label: A
predict B
A probability: 0.17204702
B probability: 0.29800123
C probability: 0.06938386
ImageName: 94188.jpg Label: C
predict A
A probability: 0.49649814
B probability: 0.04270003
C probability: 0.047948707
ImageName: 64621.jpg Label: A
predict B
A probab

A probability: 0.87676024
B probability: 0.04980753
C probability: 0.0012411097
ImageName: 95146.jpg Label: B
predict A
A probability: 0.73500603
B probability: 0.103951365
C probability: 0.0037473177
ImageName: 95247.jpg Label: B
predict A
A probability: 0.6527445
B probability: 0.112834066
C probability: 0.0028413718
ImageName: 46823.jpg Label: C
predict A
A probability: 0.999775
B probability: 0.0008204596
C probability: 0.011132392
ImageName: 79743.jpg Label: B
predict A
A probability: 0.18531412
B probability: 0.024503978
C probability: 0.17749292
ImageName: 92789.jpg Label: C
predict A
A probability: 0.77832055
B probability: 0.50175446
C probability: 7.8908735e-05
ImageName: 54900.jpg Label: B
predict A
A probability: 0.9485236
B probability: 0.007362406
C probability: 0.0046128663
ImageName: 73199.jpg Label: C
predict A
A probability: 0.7426244
B probability: 0.10639909
C probability: 0.00044321947
ImageName: 81116.jpg Label: B
predict A
A probability: 0.9995653
B probability: 

...

ImageName: 35343.jpg Label: A
predict C
A probability: 0.13904278
B probability: 0.005016352
C probability: 0.65698296
ImageName: 31281.jpg Label: B
predict A
A probability: 0.501077
B probability: 0.111315735
C probability: 0.007917394
ImageName: 47080.jpg Label: A
predict B
A probability: 0.096499205
B probability: 0.20795482
C probability: 0.005595789
ImageName: 81952.jpg Label: A
predict B
A probability: 0.093708895
B probability: 0.43215957
C probability: 0.003404167
ImageName: 50924.jpg Label: A
predict C
A probability: 0.1869655
B probability: 0.023994582
C probability: 0.36454454
ImageName: 69331.jpg Label: A
predict B
A probability: 0.14898491
B probability: 0.5331857
C probability: 0.00764999
ImageName: 97244.jpg Label: B
predict A
A probability: 0.30864933
B probability: 0.19568545
C probability: 0.031739853
ImageName: 78764.jpg Label: A
predict C
A probability: 0.2746892
B probability: 0.103720635
C probability: 0.29138023
ImageName: 51565.jpg Label: A
predict C
A probabili

predict C
A probability: 0.06728834
B probability: 0.029367799
C probability: 0.48764744
ImageName: 08523.jpg Label: A
predict B
A probability: 0.027870554
B probability: 0.7702185
C probability: 0.004206538
ImageName: 18678.jpg Label: A
predict C
A probability: 0.15712242
B probability: 0.013713753
C probability: 0.52816063
ImageName: 74262.jpg Label: B
predict C
A probability: 0.0021167244
B probability: 0.021159831
C probability: 0.9059137
ImageName: 08102.jpg Label: A
predict C
A probability: 0.20444733
B probability: 0.08337574
C probability: 0.20908348
ImageName: 38601.jpg Label: A
predict C
A probability: 0.1957118
B probability: 0.0044039288
C probability: 0.3539473
ImageName: 59630.jpg Label: A
predict B
A probability: 0.17084086
B probability: 0.28565654
C probability: 0.019970313
ImageName: 24753.jpg Label: A
predict C
A probability: 0.030871997
B probability: 0.07445247
C probability: 0.49467182
ImageName: 99707.jpg Label: B
predict C
A probability: 0.03154702
B probability

B probability: 0.12270024
C probability: 0.0008787417
ImageName: 36518.jpg Label: B
predict A
A probability: 0.9997358
B probability: 0.00056374684
C probability: 0.005740329
ImageName: 98230.jpg Label: B
predict A
A probability: 0.9470762
B probability: 0.011934973
C probability: 0.0016005818
ImageName: 09500.jpg Label: C
predict A
A probability: 0.7132286
B probability: 0.10858246
C probability: 0.0033432452
ImageName: 27086.jpg Label: B
predict A
A probability: 0.59889853
B probability: 0.023783438
C probability: 0.006808869
ImageName: 39072.jpg Label: C
predict A
A probability: 0.99957496
B probability: 9.2080954e-05
C probability: 0.00020045468
ImageName: 47885.jpg Label: C
predict A
A probability: 0.52306867
B probability: 0.00031937833
C probability: 0.4249053
ImageName: 11684.jpg Label: C
predict A
A probability: 0.99764746
B probability: 1.756337e-05
C probability: 0.053633697
ImageName: 55610.jpg Label: C
predict A
A probability: 0.9901812
B probability: 0.00025726756
C proba

A probability: 0.21269588
B probability: 0.32816067
C probability: 0.0024261097
ImageName: 85890.jpg Label: B
predict A
A probability: 0.8936274
B probability: 0.03177835
C probability: 0.0014762495
ImageName: 18959.jpg Label: B
predict A
A probability: 0.58285946
B probability: 0.20837452
C probability: 0.0019293277
ImageName: 61780.jpg Label: B
predict A
A probability: 0.6511925
B probability: 0.09763452
C probability: 0.0023738232
ImageName: 59193.jpg Label: A
predict C
A probability: 0.19283363
B probability: 0.034833867
C probability: 0.69915295
ImageName: 14678.jpg Label: B
predict A
A probability: 0.67440856
B probability: 0.051110458
C probability: 0.008739604
ImageName: 87717.jpg Label: C
predict A
A probability: 0.9999207
B probability: 2.4744088e-05
C probability: 0.0073564663
ImageName: 53254.jpg Label: B
predict A
A probability: 0.5438317
B probability: 0.10481932
C probability: 0.007949459
ImageName: 23470.jpg Label: B
predict A
A probability: 0.84775573
B probability: 0.

<Figure size 1080x720 with 0 Axes>